# Imaging Exoplanets Tutorial

We will use the `pyklip` package to subtract off the glare of the star and measure the position and brightness of the exoplanet

In [ ]:
!pip install pyklip

# Setup data directories

Before running the cells below, run the [PSF_Subtraction_Setup.ipynb](https://colab.research.google.com/drive/1AmrmAj459MwUt4kkUbfquxnf1GkyO_F2?usp=sharing) notebook to download the data. The setup notebook needs to just be run **once**.

In [ ]:
# You will be prompted to Permit this notebook to access your Google Drive files - Click on "Connect to Google Drive"
# You will then be prompted to Choose an account - click on your preferred Google account
# You will then confirm that Google Drive for desktop wants to access your Google Account - scroll to click "Allow"
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# PSF Subtraction
PSF_dir = 'SSW2024/PSF_Subtraction' #@param {type:"string"}

In [ ]:
# change to the PSF Subtraction Directory
import os

# Google top level drive dir
drive_dir = "/content/drive/MyDrive/"

# ssw_dir directory path
PSF_path = os.path.join(drive_dir, PSF_dir)

# Change to the pRT_path
os.chdir(PSF_path)


## Import Packages

In [ ]:
import numpy as np
import astropy.io.fits as fits
import scipy.ndimage
import pyklip.parallelized
import pyklip.instruments.Instrument as Instrument
import pyklip.fakes
import pyklip.kpp.metrics.crossCorr
import pyklip.kpp.stat.statPerPix_utils


import matplotlib.pylab as plt
%matplotlib inline

# Read in the necessary data

We will read in three different files:

 1. A time series of the science data. This is a 3-D image cube consisting of images of the system taken over time. The sky rotates over this period of time due to angular differential imaging. If you look really carefully, you can actually see a couple of planets! 
 2. An image of the star not blocked by the coronagraph, so we can measure the flux of the star. This is needed to measure flux ratio between the planet and the star.
 3. A 1-D array of parallactic angles. These angles specify the rotation of the sky for each frame in the 3-D image cube due to the Earth's rotation. These are needed to derotate the images so we can stack the signal of the planet together on the same pixel. 

In [ ]:
# load in the science frames for imaging the planet
with fits.open("Project_Materials_2009/center_im.fits") as hdulist:
    img_cube = hdulist[0].data # Series of images of the star system taken in time 
    exptime = hdulist[0].header['ITIME'] * hdulist[0].header['COADDS']
    
# the location of the star behind the coronagraph based on the documentaion
star_centx = (img_cube.shape[2]-1)/2
star_centy = (img_cube.shape[1]-1)/2

# load in the calibration frame to calibrate the brightness of any sources with respect to the star
with fits.open("Project_Materials_2009/median_unsat.fits") as hdulist:
    calib_frame = hdulist[0].data # image of the unsaturated star for photometric calibration
    calib_exptime = hdulist[0].header['ITIME'] * hdulist[0].header['COADDS']

# the location of the star in the calibration frame
calib_centx = (calib_frame.shape[1]-1)/2
calib_centy = (calib_frame.shape[0]-1)/2

# the parallactic angles corresponding to each frame for angular differential imaging
with fits.open("Project_Materials_2009/rotnorth.fits") as hdulist:
    rot_angles = hdulist[0].data # TODO: check sign, because negative signs are tricky
    
plt.figure()
plt.imshow(img_cube[0], cmap="inferno")
plt.xlim([star_centx-150, star_centx+150])
plt.ylim([star_centy-150, star_centy+150])
plt.title("Science Frame")

plt.figure()
plt.imshow(calib_frame, cmap="inferno")
plt.xlim([calib_centx-150, calib_centx+150])
plt.ylim([calib_centy-150, calib_centy+150])
plt.title("Calibration Frame")

# Load in the data into `pyklip`

We need to specify the location of the star in each frame, and then pass the data into the `pyklip` framework, which standardizes data from many high-contrast imaging instruments

In [ ]:
centers = np.array([[star_centx, star_centy] for _ in range(img_cube.shape[0])])

dataset = Instrument.GenericData(img_cube, centers, parangs=rot_angles)
dataset.IWA = 25
dataset.OWA = 250

# Subtract off the stellar PSF

This parallelized code runs "KLIP" (basically PCA) to remove the glare of the star. Note that is might take a couple of minutes to run -- this is a computationally intensive method. 

In [ ]:
pyklip.parallelized.klip_dataset(dataset, outputdir="./", fileprefix="epoch1", annuli=11, 
                                 subsections=1, numbasis=[1,3,5,10,20,30], maxnumbasis=100, mode="ADI",
                                 movement=1)

# Show the output image

Do you see any planets?

In [ ]:
output_img = np.nanmean(dataset.output[-1,:,0], axis=0) # combine images in time

plt.figure()
plt.imshow(output_img, cmap="inferno", vmin=np.nanpercentile(output_img, 1), vmax=np.nanpercentile(output_img, 99.7))
plt.xlim([dataset.output_centers[0,0]-200, dataset.output_centers[0,0]+200])
plt.ylim([dataset.output_centers[0,1]-200, dataset.output_centers[0,1]+200])
plt.title("Stellar PSF Subtracted Image")

# Measure the position and brighness of a planet 

The code below works for a single planet by fitting the planet to a 2-D Gaussian. This is not the most optimal method, but it is fast and good enough for demonstration purposes. 

**Note**: you need to change the `guess_x` and `guess_y` values for each planet you want to measure the properties these. These two values are an approximate (x,y) pixel location for the position of the planet in the processed image shown above. The guess needs to be fairly accurate (< 10 pixels), so you may need to zoom into portions of the image (or open the output FITS file in the FITS viewer like DS9) to come up with a good guess. 

The code cell below will measure the planet's flux, compute the relative offset from the star, and output the separation (in milliarcseconds) and position angle (in degrees) of the planet with respect to the star.

Note that the planet flux is still in arbitrary units. The cell after this one will calibrate it to the star.

In [ ]:
guess_x, guess_y = 575, 583
guess_flux = 800

peakflux, fwhm, planet_x, planet_y = pyklip.fakes.gaussfit2d(output_img, guess_x, guess_y, guesspeak=guess_flux)
print(peakflux, fwhm, planet_x, planet_y)

star_y, star_x = dataset.output_centers[0]

planet_sep_pixels = np.sqrt((planet_x - star_x)**2 + (planet_y - star_y)**2)
planet_PA = np.degrees(np.arctan2(-(planet_x - star_x), planet_y - star_y)) % 360

platescale = 9.952 # mas/pixel
planet_sep_mas = planet_sep_pixels * platescale

print(planet_sep_mas, planet_PA)

## Measure the flux ratio of the planet

We will measure the flux of the star in the calibration frame, account for the difference in exposure times, and measure the flux ratio of the planet with respect to the star. 

In [ ]:
peakflux_star, _, _, _ = pyklip.fakes.gaussfit2d(calib_frame, calib_centx, calib_centy, guesspeak=10000)

flux_ratio = (peakflux/exptime)/(peakflux_star/calib_exptime)
print(flux_ratio)

## Measure the SNR of the planet

We generally want to quantify the significance of the detection, even if we identify a planet by eye. To do this, it is common to measure the signal to noise ratio. There are two steps in measuring the signal to noise ratio.

### Step 1: Use a planet detection algorithm on the image (optional)

The most naive option is to simply take the brightest pixel in the image. However, we know that planets should appear as point sources in the image. Thus, we should leverage the fact the planet flux spans multiple pixels in an image and has a distinctive shape (the shape of the point spread function). The point spread function can be obtained from unocculted images of the star (for most coronagraphs currently used, this is a good enough approximation of the planet PSF with the coronagraph in).

Once we have the template of what a planet signal should look like, we can search for that shape across the entire image. Some options to do this include cross correlation or matched filtering (these terms are nearly identical). A simple cross correlation is the quickest to do, so we will do that here to create a cross correlation map. 

In [ ]:
# use the unocculted image of the star to create a template PSF
# we're just going to make a rough cutout to keep things simple
calib_centy_int = int(np.round(calib_centy))
calib_centx_int = int(np.round(calib_centx))
template = calib_frame[calib_centy_int-10:calib_centy_int+11, calib_centx_int-10:calib_centx_int+11]

# perform a cross correlation of the image with a planet template to get a cross correlation map
# this helps us filter out noise
cc_map = pyklip.kpp.metrics.crossCorr.calculate_cc(output_img, template, nans2zero=True)

plt.imshow(cc_map, cmap="inferno", vmin=np.nanpercentile(cc_map, 0.5), vmax=np.nanpercentile(cc_map, 99.9))
plt.xlim([dataset.output_centers[0,0]-200, dataset.output_centers[0,0]+200])
plt.ylim([dataset.output_centers[0,1]-200, dataset.output_centers[0,1]+200])

### Step 2: Compute a SNR map from your detection map

The SNR map at each pixel i can be computed from the cross correlation map (cc) and a noise map ($\sigma$). 

$$ SNR_i = \frac{cc_i}{\sigma_i} $$

To compute the noise map, we usually assume the noise is azimuthally symmetric. That is, each concentric annulus around the star has the same noise properties. This means we can take the standard deviation of the annulus to estimate the noise for all pixels in that annulus. 

Note that if there is a planet in an annulus, the noise you measure in that annulus will be artificially enhanced. There are several ways around this. The simplist is to mask out any potential planets in your image before computing the noise in each annulus. A more rigorous apporach is to compute the noise on a pixel by pixel basis, masking out a region around the pixel where a possible planet could be. However, the latter apporach drastically increases the computation time. We skip doing either here, so our SNR of our planets will be slightly underestimated here, because our noise will be overestimated. 

In [ ]:

SNR_map = pyklip.kpp.stat.statPerPix_utils.get_image_stat_map(cc_map,
                                           centroid = dataset.output_centers[0],
                                           r_step=2,
                                           Dr = 2,
                                           type = "SNR")

plt.imshow(SNR_map, cmap="inferno", vmin=-2.5, vmax=5)
plt.xlim([dataset.output_centers[0,0]-200, dataset.output_centers[0,0]+200])
plt.ylim([dataset.output_centers[0,1]-200, dataset.output_centers[0,1]+200])




In [ ]:
# read off the SNR map to get the SNR of the planet you care about.
# we will perform the "read off" using a linear interpolation to the centroid location of the planet

SNR_map[np.where(np.isnan(SNR_map))] = 0 # have to mask NaNs for this to work
planet_snr = scipy.ndimage.map_coordinates(SNR_map, [[planet_y], [planet_x]])
print(planet_snr)

# Inject Simulated Planets to Calibrate Planet Properties

Include discussion on over-subtraction and self-subtraction and reason for them. 

In [ ]:
# TODO